# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [10]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [11]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("access_token")

# Import Data

In [12]:
# Import the CSVs to Pandas DataFrames
sfo_file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(sfo_file_path, index_col="year")

nh_file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(nh_file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [13]:
# Define Panel Visualization Functions

def housing_units_per_year():
    
    """Housing Units Per Year."""
    housing_units_per_year_data = sfo_data.drop(['neighborhood','sale_price_sqr_foot','gross_rent'], 1, inplace=False).groupby(by = 'year').mean()
    housing_units_per_year = housing_units_per_year_data.hvplot.bar(x='year',
                                                           y='housing_units',
                                                           xlabel= "Year",
                                                           ylabel = "Housing Units",
                                                           ylim={370000,385000},rot=90,
                                  title=("Housing Units in San Francisco from 2010 to 2016")).opts(yformatter="%.0f")
    return housing_units_per_year

    
def average_grossrent():
    """Average Gross Rent in San Francisco Per Year."""
    average_grossrent_salesprice_data = sfo_data.groupby('year').agg({'gross_rent' : 'mean','sale_price_sqr_foot' : 'mean'})

    average_grossrent = average_grossrent_salesprice_data.hvplot(x='year',y='gross_rent',xlabel= "Year",ylabel = "Gross Rent")
    return average_grossrent

def average_salesprice():
    """Average Sales Price Per Year."""
    
    average_grossrent_salesprice_data = sfo_data.groupby('year').agg({'gross_rent' : 'mean','sale_price_sqr_foot' : 'mean'})

    average_salesprice = average_grossrent_salesprice_data.hvplot(x='year',y='sale_price_sqr_foot',xlabel= "Year",ylabel = "Average Sale Price")
    return average_salesprice

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    grouped_nh_df_data = sfo_data.groupby(['year', 'neighborhood']).mean()
    grouped_nh_df_data = grouped_nh_df_data.reset_index()
    #grouped_nh_df_data.head()    
    average_price_by_neighborhood = grouped_nh_df_data.hvplot(x='year', y='sale_price_sqr_foot', xlabel= "Year", ylabel = "Average Sale Price", groupby="neighborhood")
    return average_price_by_neighborhood

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    grouped_nh_df_data = sfo_data.groupby(['year', 'neighborhood']).mean()
    grouped_nh_df_data = grouped_nh_df_data.reset_index()
    top_10_expensive_nh = grouped_nh_df_data.nlargest(10,'sale_price_sqr_foot').set_index(np.arange(0,10))
    top_most_expensive_neighborhoods = top_10_expensive_nh.hvplot.bar(x='neighborhood', 
                                  y='sale_price_sqr_foot',
                                  xlabel= "Neighborhood", 
                                  ylabel = "Sale Price per Square Foot",
                                  ylim={1000,2300},rot=90,
                                  title=("Top 10 Expensive Neighborhood")).opts(yformatter="%.0f")
    return top_most_expensive_neighborhoods


def parallel_coordinates():
    """Parallel Coordinates Plot."""
    grouped_nh_df_data = sfo_data.groupby(['year', 'neighborhood']).mean()
    grouped_nh_df_data = grouped_nh_df_data.reset_index()
    top_10_expensive_nh = grouped_nh_df_data.nlargest(10,'sale_price_sqr_foot').set_index(np.arange(0,10))
    px.parallel_coordinates(top_10_expensive_nh, color='sale_price_sqr_foot')
    return parallel_coordinates

def parallel_categories():
    """Parallel Categories Plot."""
    grouped_nh_df_data = sfo_data.groupby(['year', 'neighborhood']).mean()
    grouped_nh_df_data = grouped_nh_df_data.reset_index()
    top_10_expensive_nh = grouped_nh_df_data.nlargest(10,'sale_price_sqr_foot').set_index(np.arange(0,10))
    
   
    data = top_10_expensive_nh[["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"]]
    px.parallel_categories(
        data,dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
        color="sale_price_sqr_foot"
    )
    return parallel_categories

def neighborhood_map():
    """Neighborhood Map"""
    df_neighborhood_locations = pd.read_csv(nh_file_path)
    df_neighborhood_locations = df_neighborhood_locations.rename(columns = {"Neighborhood" : "neighborhood"})
    df_neighborhood_locations.set_index("neighborhood")
    
    nh_df = pd.read_csv(sfo_file_path,index_col="neighborhood")
    nh_df = nh_df.drop('year' , 1, inplace=False)
    nh_df = nh_df.groupby(['neighborhood']).mean()
    
    joined_df = df_neighborhood_locations.join(nh_df, on="neighborhood")
    joined_df = joined_df.dropna()
    # Plot Data
    neighborhood_map = px.scatter_mapbox(
        joined_df,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="gross_rent",
        zoom=10
    )
    return neighborhood_map

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [14]:
# Create panels to structure the layout of the dashboard
averagesales_rent_column = pn.Column(
    "## Housing Units Per Year,Average Gross Rent in San Francisco Per Year,Average Sales Price Per Year" ,
    housing_units_per_year(), 
    average_grossrent(),
    average_salesprice()
)

neighborhood_column = pn.Column(
    "## Average Prices by Neighborhood and Top 10 Most Expensive Neighborhoods",
    average_price_by_neighborhood(),
    top_most_expensive_neighborhoods()
#   neighborhood_map()
)


#parallelplot_column = pn.Column(
#    "## Parallel Plots",
#    parallel_coordinates()
#    parallel_categories()
#)


## Serve the Panel Dashboard

In [17]:

dashboard = pn.Tabs(
    ("SF Housing Analysis", averagesales_rent_column),
    ("SF Neighborhood", neighborhood_column)    
#    ("SF Parallel Plots", parallelplot_column)
)
dashboard.show()

Launching server at http://localhost:51675
